# 基于概率论的分类方法:朴素贝叶斯 

### 使用Python进行文本分类


计算条件概率的方法:贝叶斯准则
$$
p(c|x) = \frac{p(x|c)p(c)}{p(x)}
$$

In [1]:
#词表到向量的转换函数
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]  #1 代表侮辱性文字
    return postingList,classVec        

In [2]:
def createVocabList(dataSet):
    vocabSet = set()
    for document in dataSet:
        vocabSet = vocabSet|set(document)
    return list(vocabSet)

In [3]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec        

In [4]:
listOPosts, listClasses = loadDataSet()
listOPosts
listOPosts[0]

['my', 'dog', 'has', 'flea', 'problems', 'help', 'please']

In [5]:
myVocabList = createVocabList(listOPosts)
print(myVocabList)

['my', 'him', 'garbage', 'ate', 'stop', 'not', 'cute', 'worthless', 'maybe', 'licks', 'problems', 'stupid', 'love', 'so', 'steak', 'flea', 'is', 'park', 'I', 'dalmation', 'how', 'buying', 'help', 'quit', 'food', 'mr', 'has', 'dog', 'please', 'take', 'to', 'posting']


In [6]:
print(setOfWords2Vec(myVocabList, listOPosts[1]))

[0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0]


In [20]:
import numpy as np
import cmath
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.zeros(numWords); p1Num = np.zeros(numWords)
    p0Denom = 0.0; p1Denom = 0.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = cmath.log(p1Num/p1Denom)
    p0Vect = cmath.log(p0Num/p0Denom)
    return p0Vect, p1Vect, pAbusive

In [21]:
listOPosts, listClasses = loadDataSet()

In [22]:
myVocabList = createVocabList(listOPosts)
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

In [23]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

TypeError: only length-1 arrays can be converted to Python scalars